<h1>Build feature and output arrays

In [8]:
import numpy as np
import pandas as pd
import json
from sklearn.preprocessing import OneHotEncoder

# lists for all data
all_turbine_types = []
all_hub_heights = []
all_capacities = []
all_commissioning_dates = []
all_production_data = []

with open(f"data/WPPs+production+wind.json", "r", encoding="utf-8") as file:
    WPP_production_wind = json.load(file)

# collect data
for wpp in WPP_production_wind:
    all_turbine_types.append([wpp["Turbine"] if pd.notna(wpp["Turbine"]) else "nan" for wpp in WPP_production_wind])
    all_hub_heights.append(wpp["Hub_height"] if not pd.isna(wpp["Hub_height"]) else 100)
    all_capacities.append(wpp["Capacity"])
    all_commissioning_dates.append("2015/06" if wpp["Commission_date"] == "nan" else f"{wpp['Commission_date']}/06" if isinstance(wpp["Commission_date"], str) and "/" not in wpp["Commission_date"] else wpp["Commission_date"])
    all_production_data.append(wpp["Production"])

# One-Hot-Encoding for turbine types
encoder = OneHotEncoder(sparse_output=False)
turbine_types_onehot = encoder.fit_transform(np.array(all_turbine_types).reshape(-1, 1))

# convert to datetime
standardised_dates = pd.to_datetime(all_commissioning_dates, format='%Y/%m')

# calculate age
current_date = pd.Timestamp("2024-12-01")
ages = current_date.year * 12 + current_date.month - (standardised_dates.year * 12 + standardised_dates.month)

# create combined features and output lists
combined_features_raw = []
output_raw = []

# convert data in feature arrays
for idx, production_data in enumerate(all_production_data):
    num_rows = len(production_data)

    # repetitions for common features
    turbine_type_repeated = np.tile(turbine_types_onehot[idx], (num_rows, 1))
    hub_height_repeated = np.full((num_rows, 1), all_hub_heights[idx])
    age_repeated = np.full((num_rows, 1), ages[idx])

    # extract production values and wind speeds
    production_values = np.array([entry[1] for entry in production_data]).reshape(-1, 1) / all_capacities[idx]
    wind_speeds = np.array([entry[2] for entry in production_data]).reshape(-1, 1)

    # combine all features
    combined_chunk = np.hstack((
        turbine_type_repeated,
        hub_height_repeated,
        age_repeated,
        wind_speeds
    ))

    # add the data
    combined_features_raw.append(combined_chunk)
    output_raw.append(production_values)

np.save("turbine_types_order.npy", encoder.categories_[0])

# combine all data chunks to one array
combined_features_raw = np.vstack(combined_features_raw)
output_raw = np.vstack(output_raw)

# round all values to two decimal places
combined_features_raw = np.round(combined_features_raw, decimals=4)
output_raw = np.round(output_raw, decimals=4)

In [9]:
mask = output_raw.flatten() > 0.01

# Apply the mask to filter the subsets
combined_features_filtered = combined_features_raw[mask]
output_filtered = output_raw[mask]

<h1>Scale feature vector and define Dataset

In [10]:
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
import torch
import joblib

combined_features = combined_features_filtered.copy()
output = output_filtered.copy()

# Separate Scaler für jedes Feature
scaler_wind = StandardScaler()
scaler_ages = StandardScaler()
scaler_hub_heights = StandardScaler()

# Skalieren der einzelnen Features
combined_features[:, -1] = scaler_wind.fit_transform(combined_features[:, -1].reshape(-1, 1)).flatten() # scale wind speeds
combined_features[:, -2] = scaler_ages.fit_transform(combined_features[:, -2].reshape(-1, 1)).flatten()  # scale ages
combined_features[:, -3] = scaler_hub_heights.fit_transform(combined_features[:, -3].reshape(-1, 1)).flatten()  # scale hub heights

# Speichere alle Scaler in einem Dictionary
scalers = {
    "winds": scaler_wind,
    "ages": scaler_ages,
    "hub_heights": scaler_hub_heights,
}

# Speichere das Dictionary mit Joblib
joblib.dump(scalers, "scalers.pkl")

# Dataset-Klasse für PyTorch
class WindPowerDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        x = self.features[index]
        y = self.targets[index]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Erstellung der PyTorch-Datasets
dataset = WindPowerDataset(combined_features, output)

<h1>Define Model

In [4]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_size, use_dropout=False, dropout_rate=0.3, 
                 use_batch_norm=False, activation_fn=nn.ReLU):
        super(MLP, self).__init__()

        layers = []

        # Erste Schicht
        layers.append(nn.Linear(input_size, 256))
        if use_batch_norm:
            layers.append(nn.BatchNorm1d(256))
        layers.append(activation_fn())

        # Zweite Schicht
        layers.append(nn.Linear(256, 128))
        if use_batch_norm:
            layers.append(nn.BatchNorm1d(128))
        layers.append(activation_fn())

        # Dritte Schicht
        layers.append(nn.Linear(128, 64))
        if use_batch_norm:
            layers.append(nn.BatchNorm1d(64))
        layers.append(activation_fn())

        # Dropout nach der letzten versteckten Schicht (optional)
        if use_dropout:
            layers.append(nn.Dropout(dropout_rate))

        # Ausgabeschicht
        layers.append(nn.Linear(64, 1))

        # Modell zusammenstellen
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

<h1>1. Hyperparameter search: Training, Validation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
import numpy as np
import optuna
import time
import pynvml
import psutil

# Ressourcenüberwachung initialisieren
gpu_used = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda"):
    pynvml.nvmlInit()
    gpu_handle = pynvml.nvmlDeviceGetHandleByIndex(0)  # GPU 0 verwenden

n_splits = 2  # Anzahl der Folds für Kreuzvalidierung

def system_info():
    print("PyTorch Version:", torch.__version__)
    print("CUDA Available:", torch.cuda.is_available())
    print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    if torch.cuda.is_available():
        print("\nCUDA Details:")
        print("CUDA Version:", torch.version.cuda)
        print("Number of GPUs:", torch.cuda.device_count())
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"  - Compute Capability: {torch.cuda.get_device_capability(i)}")
            print(f"  - Memory Allocated: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
            print(f"  - Memory Cached: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")

if __name__ == "__main__":
    system_info()

# Bewertungsfunktion
def objective(trial):
    original_batch_size = trial.suggest_int("batch_size", 16, 128)
    batch_size = int(2 ** round(np.log2(original_batch_size)))  # Transformierte Batch-Größe
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    number_epochs = trial.suggest_int("number_epochs", 10, 100)
    use_dropout = trial.suggest_categorical("use_dropout", [True, False])
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    use_batch_norm = trial.suggest_categorical("use_batch_norm", [True, False])

    # Speichern der transformierten Batch-Größe als Attribut
    trial.set_user_attr("transformed_batch_size", batch_size)

    print(f"Evaluating: batch_size={batch_size}, lr={lr:.5f}, number_epochs={number_epochs}, "
          f"use_dropout={use_dropout}, dropout_rate={dropout_rate}, use_batch_norm={use_batch_norm}")

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    len_dataset = len(dataset)

    input_size = combined_features.shape[1]
    avg_val_loss = 0.0  # Durchschnittlicher Validierungsverlust
    start_time = time.time()  # Zeitmessung starten
    max_memory_usage = 0  # Maximale Speicher-Auslastung

    for fold, (train_idx, val_idx) in enumerate(kf.split(range(len_dataset)), 1):
        print(f"  Fold {fold}/{kf.n_splits}")

        model = torch.jit.script(MLP(input_size=input_size, use_dropout=use_dropout, dropout_rate=dropout_rate, use_batch_norm=use_batch_norm)).to(device)

        train_fold_dataset = Subset(dataset, train_idx)
        val_fold_dataset = Subset(dataset, val_idx)

        train_loader = DataLoader(train_fold_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_fold_dataset, batch_size=batch_size, shuffle=False)

        criterion = nn.HuberLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        for epoch in range(number_epochs):
            model.train()
            for batch_x, batch_y in train_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if device == torch.device("cuda"):
                    memory_info = pynvml.nvmlDeviceGetMemoryInfo(gpu_handle)
                    max_memory_usage = max(max_memory_usage, memory_info.used / 1024 ** 2)  # MB
                    global gpu_used
                    gpu_used = True
                else:
                    max_memory_usage = max(max_memory_usage, psutil.virtual_memory().used / 1024 ** 2)  # MB

        model.eval() # deactivates Batch normalisation and Dropout
        fold_val_loss = 0.0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                val_outputs = model(batch_x)
                fold_val_loss += criterion(val_outputs, batch_y).item()
                
        avg_val_loss += fold_val_loss / len(val_loader)

    avg_val_loss /= kf.n_splits
    elapsed_time = time.time() - start_time

    weighted_score = 0.7 * avg_val_loss + 0.15 * elapsed_time + 0.15 * max_memory_usage

    trial.set_user_attr("resource_usage", max_memory_usage)
    trial.set_user_attr("elapsed_time", elapsed_time)
    trial.set_user_attr("avg_val_loss", avg_val_loss)
    trial.set_user_attr("weighted_score", weighted_score)

    return weighted_score

# Optuna-Optimierung starten
best_val_loss = float("inf")
best_params = None

study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=15, show_progress_bar=True)

print("\nBeste Parameterkombination:")
print(study.best_params)

with open("best_params.json", "w") as f:
    json.dump(study.best_params, f)

# Ausgabe des besten Validierungsverlusts
best_trial = study.best_trial
print(f"Bester Validierungsverlust: {best_trial.user_attrs['avg_val_loss']}")

for trial in study.trials:
    print(trial)

# Debugging: Überprüfung, ob GPU verwendet wurde
if gpu_used:
    print("GPU wurde erfolgreich während des Trainings verwendet.")
else:
    print("GPU wurde nicht verwendet.")

[I 2025-01-13 09:22:06,436] A new study created in memory with name: no-name-b97103b7-ef9f-4179-978e-933285641b5e


PyTorch Version: 2.5.1+cpu
CUDA Available: False
Device: cpu


  0%|          | 0/20 [00:00<?, ?it/s]

Evaluating: batch_size=64, lr=0.00030, number_epochs=51, use_dropout=False, dropout_rate=0.4403619936961537, use_batch_norm=True
  Fold 1/4
[W 2025-01-13 09:22:23,057] Trial 0 failed with parameters: {'batch_size': 51, 'lr': 0.00029627547762384323, 'number_epochs': 51, 'use_dropout': False, 'dropout_rate': 0.4403619936961537, 'use_batch_norm': True} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\alexa\anaconda3\envs\webapp_env_conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\alexa\AppData\Local\Temp\ipykernel_42312\903217237.py", line 84, in objective
    optimizer.step()
  File "c:\Users\alexa\anaconda3\envs\webapp_env_conda\Lib\site-packages\torch\optim\optimizer.py", line 487, in wrapper
    out = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\alexa\anaconda3\envs\webapp_env_conda\Lib\site-pa

KeyboardInterrupt: 

<h1>2. With best hyperparameters: Training and Testing

In [ ]:
# from sklearn.model_selection import train_test_split
# from torch.utils.tensorboard import SummaryWriter
# from torch.nn import HuberLoss, MSELoss, L1Loss
# import subprocess
# import platform
# import os
# import shutil

# log_dir = "runs"

# # TensorBoard-Prozess beenden
# try:
#     if platform.system() == "Windows":
#         subprocess.run(["taskkill", "/IM", "tensorboard.exe", "/F"], check=True)
#     else:
#         subprocess.run(["pkill", "-f", "tensorboard"], check=True)
#     print("Terminated TensorBoard process")
# except subprocess.CalledProcessError:
#     print("No TensorBoard process found or could not be terminated")

# # Log-Verzeichnis löschen
# if os.path.exists(log_dir):
#     shutil.rmtree(log_dir)
# os.makedirs(log_dir, exist_ok=True)

# subprocess.Popen(["tensorboard", "--logdir=runs", "--bind_all"])
# print("TensorBoard started.")

# # TensorBoard-Writer starten
# writer = SummaryWriter(f"{log_dir}/final_training")

# # Daten splitten
# train_x, test_x, train_y, test_y = train_test_split(combined_features, output, test_size=0.2, shuffle=True, random_state=42)

# with open("model1/best_params.json", "r") as f:
#     best_params = json.load(f)

# # Modell initialisieren
# model = MLP(
#     input_size=train_x.shape[1],
#     use_dropout=best_params["use_dropout"],
#     dropout_rate=best_params["dropout_rate"],
#     use_batch_norm=best_params["use_batch_norm"]
# ).to(device)

# # Visualisierung des Modells
# example_input = torch.randn(best_params["batch_size"], train_x.shape[1]).to(device)
# writer.add_graph(model, example_input)

# # Trainings-Konfiguration
# criterion = HuberLoss()
# optimizer = optim.Adam(model.parameters(), lr=best_params["lr"])

# # Daten-Loader
# train_loader = DataLoader(list(zip(train_x, train_y)), batch_size=best_params["batch_size"], shuffle=True)
# test_loader = DataLoader(list(zip(test_x, test_y)), batch_size=best_params["batch_size"], shuffle=False)

# # Training
# num_epochs = best_params["number_epochs"]

# for epoch in range(num_epochs):
#     model.train()
#     train_loss = 0

#     for batch_x, batch_y in train_loader:
#         batch_x, batch_y = batch_x.to(device), batch_y.to(device)
#         outputs = model(batch_x)
#         loss = criterion(outputs, batch_y)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()

#     train_loss /= len(train_loader)
#     writer.add_scalar("Training Loss", train_loss, epoch)

# # Testen
# model.eval()
# mae_criterion = L1Loss()
# mse_criterion = MSELoss()
# huber_criterion = HuberLoss()

# test_loss_mae, test_loss_mse, test_loss_huber = 0, 0, 0

# with torch.no_grad():
#     for batch_x, batch_y in test_loader:
#         batch_x, batch_y = batch_x.to(device), batch_y.to(device)
#         preds = model(batch_x)
        
#         test_loss_mae += mae_criterion(preds, batch_y).item()
#         test_loss_mse += mse_criterion(preds, batch_y).item()
#         test_loss_huber += huber_criterion(preds, batch_y).item()

# test_loss_mae /= len(test_loader)
# test_loss_mse /= len(test_loader)
# test_loss_huber /= len(test_loader)

# # Ergebnisse anzeigen
# print(f"Endgültiger MAE: {test_loss_mae:.4f}")
# print(f"Endgültiger MSE: {test_loss_mse:.4f}")
# print(f"Endgültiger Huber Loss: {test_loss_huber:.4f}")

# # TensorBoard-Writer schließen
# writer.close()

<h1>3. With all data: Training for deployment

In [ ]:
# # Modell initialisieren
# model = MLP(
#     input_size=combined_features.shape[1],
#     use_dropout=best_params["use_dropout"],
#     dropout_rate=best_params["dropout_rate"],
#     use_batch_norm=best_params["use_batch_norm"]
# ).to(device)

# # Trainings-Konfiguration
# criterion = HuberLoss()
# optimizer = optim.Adam(model.parameters(), lr=best_params["lr"])

# # Daten-Loader für alle Trainingsdaten
# train_loader = DataLoader(list(zip(combined_features, output)), batch_size=best_params["batch_size"], shuffle=True)

# # Training
# num_epochs = best_params["number_epochs"]

# for epoch in range(num_epochs):
#     model.train()
#     train_loss = 0

#     for batch_x, batch_y in train_loader:
#         batch_x, batch_y = batch_x.to(device), batch_y.to(device)
#         outputs = model(batch_x)
#         loss = criterion(outputs, batch_y)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()

#     train_loss /= len(train_loader)
#     writer.add_scalar("Final Training Loss", train_loss, epoch)

# # Modell speichern
# torch.save(model.state_dict(), "trained_parameters.pth")
# print("Modell für Deployment gespeichert!")

KeyError: 'use_dropout'